# Example
use diamonds dataset to predict **price** in terms of other variables. For this, 
a. estimate the parameters
b. select the imporant input features 
c. write down the predictive model 

since price is practically continous therefore we need to apply linear regression:
to do the following tasks:
1.   parameter learning
2.   feature selection
3.   predictive model (use steps 1 and 2 and link function)
4.   prediction 
5.   Evaluate the performance of prediction (split the dataset into trainset and testset)

We can do Linear regression using 2 different ways:
1.  using statistical learning
2.  using sklearn

data modeling:
1. import the function of the model
2. create the model
3.  fit the model
4. prediction using the model

# Statistical way to recall GLM

In [16]:
#Import neccessary library
from statistics import mean
import statsmodels.api as sm
from statsmodels.api import GLM, add_constant
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Preparation

In [17]:
data = read_csv('dataset/diamonds.csv')

X = data.drop(['price', 'Unnamed: 0'], axis=1)  # Input variables
y = data['price']  # Output variable

OE = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Ideal', 'Premium']])

data_cut_OE = OE.fit_transform(X[['cut']])
data_cut_DF = DataFrame(data_cut_OE)
data_cut_DF.columns = ['encoded cut']

OHE = OneHotEncoder(handle_unknown='ignore')

data_color_OHE = OHE.fit_transform(X[['color']])
data_color_DF = DataFrame(data_color_OHE.toarray())
data_color_DF.columns = OHE.get_feature_names_out()

data_clarity_OHE = OHE.fit_transform(X[['clarity']])
data_clarity_DF = DataFrame(data_clarity_OHE.toarray())
data_clarity_DF.columns = OHE.get_feature_names_out()

# Merging multiple DataFrames

In [18]:
X_binary = concat([data_color_DF, data_clarity_DF], axis=1) # Excluding data_cut_DF since it is ordinal
X_scalable = X[['carat', 'depth', 'table', 'length_mm', 'width_mm', 'depth_mm']]  # Orginal numeric columns
X_scalable = concat([X_scalable, data_cut_DF], axis=1)  # Including encoded ordinal columns

# Applying StandardScaler

In [19]:
X_scaled = StandardScaler().fit_transform(X_scalable)
X_scaled_DF = DataFrame(X_scaled)
X_scaled_DF.columns = X_scalable.columns

In [20]:
X_PREP = concat([X_scaled_DF, X_binary], axis=1)  # Prepared Data

# Data Modelling for feature selection / reduced model

In [21]:
X = add_constant(X_PREP)  # Ading constant column to handle bias; 
model_full = GLM(y, X, family=sm.families.Gaussian(sm.families.links.log()))
res = model_full.fit()  # Fitting the model
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  price   No. Observations:                  859
Model:                            GLM   Df Residuals:                      838
Model Family:                Gaussian   Df Model:                           20
Link Function:                    log   Scale:                          50649.
Method:                          IRLS   Log-Likelihood:                -5861.0
Date:                Wed, 08 Mar 2023   Deviance:                   4.2444e+07
Time:                        02:11:25   Pearson chi2:                 4.24e+07
No. Iterations:                    22   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            6.0650      0.005   1159.381      0.000       6.055       6.075
carat           -0.5882      0.039    -14.960      0.000      -0.665      -0.511
depth           -0.0430      0.045     -0.958      0.338      -0.131       0.045
table            0.0243      0.004      6.350      0.000       0.017       0.032
length_mm        0.0201      0.099      0.203      0.839      -0.174       0.215
width_mm         0.3744      0.092      4.087      0.000       0.195       0.554
depth_mm         0.7281      0.171      4.270      0.000       0.394       1.062
encoded cut      0.0122      0.003      3.517      0.000       0.005       0.019
color_D          0.9806      0.009    111.662      0.000       0.963       0.998
color_E          0.9480      0.007    139.634      0.000       0.935       0.961
color_F          0.9357      0.007    138.875      0.000       0.923       0.949
color_G          0.8831      0.008    113.083      0.000       0.868       0.898
color_H          0.8194      0.009     94.732      0.000       0.802       0.836
color_I          0.7579      0.011     67.488      0.000       0.736       0.780
color_J          0.7402      0.016     44.962      0.000       0.708       0.772
clarity_I1       0.4579      0.023     20.128      0.000       0.413       0.502
clarity_IF       0.9950      0.022     45.326      0.000       0.952       1.038
clarity_SI1      0.6948      0.007    101.299      0.000       0.681       0.708
clarity_SI2      0.5914      0.009     62.388      0.000       0.573       0.610
clarity_VS1      0.7624      0.008     91.078      0.000       0.746       0.779
clarity_VS2      0.7443      0.007    105.410      0.000       0.730       0.758
clarity_VVS1     0.9429      0.014     67.375      0.000       0.916       0.970
clarity_VVS2     0.8762      0.012     72.099      0.000       0.852       0.900
================================================================================
"""

In [22]:
pvalues = dict(res.pvalues)  # Identifying the columns to drop
for x in pvalues:
    if pvalues[x] >= 0.05:
        print(x)

depth
length_mm


In [23]:
X_red = X.drop(['depth', 'length_mm'], axis=1)

Predictive Model:  yhat=5.455245+0.001909*ZN+0.022373*INDUS+0.028275*RAD-0.002238*MEDV    

In [24]:
rmse_full = []
rmse_red = []
for i in range(20):
    X = add_constant(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    #*************Full Data Modelling*************
    
    # Creating the Linear Regression Model
    model_full_train = GLM(y_train, X_train, family=sm.families.Gaussian(sm.families.links.log()))
    model_full = model_full_train.fit()  # Fitting the model
    model_full.summary()

    # Predicting using Test-set
    pred_full = model_full.predict(X_test)
    rmse1 = sqrt(mean_squared_error(y_test, pred_full))
    rmse_full.append(rmse1)
    
    #*************Reduced Data Modelling*************
    
    # Creating the Linear Regression Model
    X = add_constant(X_red)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model_reduced_train = GLM(y_train, X_train, family=sm.families.Gaussian(sm.families.links.log()))
    model_reduced = model_reduced_train.fit()
    
    # Predicting using Test-set
    pred_red = model_reduced.predict(X_test)
    rmse2 = sqrt(mean_squared_error(y_test, pred_red))
    rmse_red.append(rmse2)

[mean(rmse_full), mean(rmse_red)]

[228.95992067970442, 230.3011580919856]

# Deployment phase 

In [25]:
X = add_constant(X_red)
model_train = GLM(y, X, family=sm.families.Gaussian(sm.families.links.log()))  # create the linear reg model using
red_model = model_train.fit() # fit the model

# Predicting using testset

In [26]:
new = X.tail(1)
pred_red = red_model.predict(new)
print(pred_red)
y.tail(1)

858    2824.587072
dtype: float64


858    2871
Name: price, dtype: int64

# Statistical & sklearn way to recall GLM

In [40]:
# Import neccessary library
from statistics import mean
import statsmodels.api as sm
from statsmodels.api import GLM, add_constant
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Preparation

In [41]:
data = read_csv('dataset/diamonds.csv')

X = data.drop(['price', 'Unnamed: 0'], axis=1)  # Input variables
y = data['price']  # Output variable

OE = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Ideal', 'Premium']])
data_cut_OE = OE.fit_transform(X[['cut']])
data_cut_DF = DataFrame(data_cut_OE)
data_cut_DF.columns = ['encoded cut']

OHE = OneHotEncoder(handle_unknown='ignore')

data_color_OHE = OHE.fit_transform(X[['color']])
data_color_DF = DataFrame(data_color_OHE.toarray())
data_color_DF.columns = OHE.get_feature_names_out()

data_clarity_OHE = OHE.fit_transform(X[['clarity']])
data_clarity_DF = DataFrame(data_clarity_OHE.toarray())
data_clarity_DF.columns = OHE.get_feature_names_out()

# Merging multiple DataFrames

In [42]:
X_binary = concat([data_color_DF, data_clarity_DF], axis=1) # Excluding data_cut_DF since it is ordinal
X_scalable = X[['carat', 'depth', 'table', 'length_mm', 'width_mm', 'depth_mm']]  # Orginal numeric columns
X_scalable = concat([X_scalable, data_cut_DF], axis=1) # Including encoded ordinal columns

# Applying standard scalar

In [44]:
X_scaled = StandardScaler().fit_transform(X_scalable)
X_scaled_DF = DataFrame(X_scaled)
X_scaled_DF.columns = X_scalable.columns

In [45]:
X_PREP = concat([X_scaled_DF, X_binary], axis=1)  # Prepared Data

# Data Modelling for feature selection / reduced model

In [46]:
X = add_constant(X_PREP)  # Ading constant column to handle bias
model_full = GLM(y, X, family=sm.families.Gaussian(sm.families.links.log()))
res = model_full.fit()  # Fitting the model
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  price   No. Observations:                  859
Model:                            GLM   Df Residuals:                      838
Model Family:                Gaussian   Df Model:                           20
Link Function:                    log   Scale:                          50649.
Method:                          IRLS   Log-Likelihood:                -5861.0
Date:                Wed, 08 Mar 2023   Deviance:                   4.2444e+07
Time:                        02:23:04   Pearson chi2:                 4.24e+07
No. Iterations:                    22   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            6.0650      0.005   1159.381      0.000       6.055       6.075
carat           -0.5882      0.039    -14.960      0.000      -0.665      -0.511
depth           -0.0430      0.045     -0.958      0.338      -0.131       0.045
table            0.0243      0.004      6.350      0.000       0.017       0.032
length_mm        0.0201      0.099      0.203      0.839      -0.174       0.215
width_mm         0.3744      0.092      4.087      0.000       0.195       0.554
depth_mm         0.7281      0.171      4.270      0.000       0.394       1.062
encoded cut      0.0122      0.003      3.517      0.000       0.005       0.019
color_D          0.9806      0.009    111.662      0.000       0.963       0.998
color_E          0.9480      0.007    139.634      0.000       0.935       0.961
color_F          0.9357      0.007    138.875      0.000       0.923       0.949
color_G          0.8831      0.008    113.083      0.000       0.868       0.898
color_H          0.8194      0.009     94.732      0.000       0.802       0.836
color_I          0.7579      0.011     67.488      0.000       0.736       0.780
color_J          0.7402      0.016     44.962      0.000       0.708       0.772
clarity_I1       0.4579      0.023     20.128      0.000       0.413       0.502
clarity_IF       0.9950      0.022     45.326      0.000       0.952       1.038
clarity_SI1      0.6948      0.007    101.299      0.000       0.681       0.708
clarity_SI2      0.5914      0.009     62.388      0.000       0.573       0.610
clarity_VS1      0.7624      0.008     91.078      0.000       0.746       0.779
clarity_VS2      0.7443      0.007    105.410      0.000       0.730       0.758
clarity_VVS1     0.9429      0.014     67.375      0.000       0.916       0.970
clarity_VVS2     0.8762      0.012     72.099      0.000       0.852       0.900
================================================================================
"""

In [47]:
pvalues = dict(res.pvalues)  # Identifying the columns to drop
for x in pvalues:
    if pvalues[x] >= 0.05:
        print(x)

depth
length_mm


In [48]:
X_red = X.drop(['depth', 'length_mm'], axis=1)

Predictive Model:  yhat=5.455245+0.001909*ZN+0.022373*INDUS+0.028275*RAD-0.002238*MEDV

In [49]:
rmse_st_full = []
rmse_st_red = []
rmse_skl_full = []
rmse_skl_red = []

model_skl = LinearRegression()
for i in range(20):
    #************************* Full Data Modelling  ***************************#
    X = add_constant(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # statsmodels
    model_st_full_train = GLM(y_train, X_train, family=sm.families.Gaussian(sm.families.links.log()))
    model_st_full = model_st_full_train.fit()  # Fitting the model

    # Predicting using Test-set
    pred_st_full = model_st_full.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, pred_st_full))
    rmse_st_full.append(rmse)
    
    # sklearn
    model_skl.fit(X_train, y_train)
    
    # Predicting using Test-set
    pred_skl_full = model_skl.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, pred_skl_full))
    rmse_skl_full.append(rmse)
    
    #************************* Reduced Data Modelling  ***************************#
    X = add_constant(X_red)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # statsmodels
    model_st_red_train = GLM(y_train, X_train, family=sm.families.Gaussian(sm.families.links.log()))  # create the linear reg model using
    model_st_red = model_st_red_train.fit()
    
    # Predicting using Test-set
    pred_red = model_reduced.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, pred_red))
    rmse_st_red.append(rmse)
    
    # sklearn
    model_skl.fit(X_train, y_train)
    
    # Predicting using Test-set
    pred_skl_red = model_skl.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, pred_skl_red))
    rmse_skl_red.append(rmse)
    

[mean(rmse_st_full), mean(rmse_st_red), mean(rmse_skl_full), mean(rmse_skl_red)]

[233.14823973272647,
 224.86136882050604,
 163.87127117672674,
 162.87693038257981]

# Deployment Phase

In [50]:
X = add_constant(X_red)
model_skl_red_dep = model_skl.fit(X, y)

# Predicting using Test-set
# Predict the tax using the linear regression for the last row in the boston_house-prices dataset

In [51]:
pred_st_red_dep = model_skl_red_dep.predict(X.tail(1))
print(pred_st_red_dep)
y.tail(1)

[2809.03468253]


858    2871
Name: price, dtype: int64

# Logistic regression
Use boston_house price dataset, 
1.   Consider CHAS as the output variable 
2.   the remaining variables are inputs 
apply logistic regression to do the following tasks:
*  parameter estimation 
*   attribute selection 
*  predictive model
*   prediction

In [39]:
# Import neccessary library
from statistics import mean
import statsmodels.api as sm
from statsmodels.api import GLM, add_constant
from pandas import read_csv, DataFrame, concat
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Preparation

In [40]:
data = read_csv('dataset/boston_house_prices.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [41]:
data['CHAS'].unique()

array([0, 1])

In [42]:
X = data.drop('CHAS', axis=1)  # input
y = data['CHAS']   # output

# Data Modelling for feature selection / reduced model

# Logistic regression using statsmodels

In [43]:
X = sm.add_constant(X)  # add intercept
model_logistic = sm.GLM(y, X, family=sm.families.Binomial())
res = model_logistic.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   CHAS   No. Observations:                  506
Model:                            GLM   Df Residuals:                      492
Model Family:                Binomial   Df Model:                           13
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -105.75
Date:                Wed, 08 Mar 2023   Deviance:                       211.49
Time:                        05:45:32   Pearson chi2:                     337.
No. Iterations:                     8   Pseudo R-squ. (CS):            0.08150
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5810      4.939     -0.725      0.468     -13.262       6.099
CRIM          -0.2717      0.152     -1.792      0.073      -0.569       0.026
ZN            -0.0018      0.014     -0.129      0.898      -0.029       0.025
INDUS          0.0952      0.042      2.259      0.024       0.013       0.178
NOX            3.1623      3.174      0.996      0.319      -3.059       9.383
RM            -0.1114      0.328     -0.340      0.734      -0.754       0.531
AGE            0.0083      0.012      0.665      0.506      -0.016       0.033
DIS           -0.0221      0.216     -0.102      0.918      -0.445       0.401
RAD            0.2191      0.078      2.799      0.005       0.066       0.373
TAX           -0.0087      0.004     -2.265      0.023      -0.016      -0.001
PTRATIO       -0.1467      0.129     -1.136      0.256      -0.400       0.106
B              0.0024      0.004      0.636      0.525      -0.005       0.010
LSTAT          0.0244      0.050      0.491      0.624      -0.073       0.122
MEDV           0.0693      0.032      2.138      0.032       0.006       0.133
==============================================================================
"""

In [44]:
pvalues = dict(res.pvalues)  # Identifying the columns to drop
for x in pvalues:
    if pvalues[x] >= 0.05:
        print(x)

const
CRIM
ZN
NOX
RM
AGE
DIS
PTRATIO
B
LSTAT


In [45]:
X_red = X.drop(['const', 'CRIM', 'ZN', 'NOX', 'RM', 'AGE', 'DIS', 'PTRATIO', 'B', 'LSTAT'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)  # testset is 10%

Predictive model based on above results:

that_i = 0.0952 * INDUS_i + 0.2191 * RAD_i - 0.0087 * TAX_i + 0.0693 * MEDV_i

1.   phat_i = 1 / (1 + exp(-that_i))
2.   yhat_i = 1 if phat_i >= 0.5 else yhat_i = 0

In [46]:
model_st_red_train = GLM(y_train, X_train, family=sm.families.Binomial()) # logistic regression
model_st_red = model_st_red_train.fit()

In [47]:
prediction_st_raw = model_st_red.predict(X_test)
prediction_st_raw

224    0.245663
137    0.045313
453    0.067303
303    0.067967
254    0.005386
37     0.029412
442    0.112732
417    0.000672
16     0.022943
209    0.024132
126    0.208278
157    0.383280
196    0.020893
266    0.161863
404    0.000053
399    0.013794
116    0.023814
127    0.037455
134    0.025365
201    0.007439
503    0.053354
161    0.534502
287    0.025959
73     0.026413
439    0.035295
325    0.039428
112    0.018129
310    0.017442
14     0.016644
230    0.053991
27     0.009174
291    0.053341
479    0.018896
102    0.007754
124    0.321740
376    0.010298
248    0.018770
237    0.111496
354    0.002151
334    0.037681
153    0.216423
392    0.014511
218    0.113605
458    0.048127
357    0.211703
101    0.023423
269    0.027910
211    0.023232
348    0.015260
103    0.012016
349    0.003932
dtype: float64

# Converting predictions to binary predicted values

In [48]:
prediction_st = [1 if prediction > 0.5 else 0 for prediction in prediction_st_raw]
df = DataFrame({"Prediction": prediction_st, "Actual": y_test})
df.head()


,Prediction,Actual
224,0,0
137,0,0
453,0,0
303,0,0
254,0,0


# Logistic regression using sklearn

In [53]:
model_skl = LogisticRegression(max_iter=10000)
model_skl = model_skl.fit(X_train, y_train)
prediction_skl = model_skl.predict(X_test)

In [54]:
df = DataFrame({"Prediction_st": prediction_st, "Prediction_skl": prediction_skl, "Actual": y_test})
df.head()

,Prediction_st,Prediction_skl,Actual
224,0,0,0
137,0,0,0
453,0,0,0
303,0,0,0
254,0,0,0


# Compute recall and accuracy

In [28]:
from sklearn.metrics import accuracy_score, recall_score
# Prediction_st
acc_st = accuracy_score(y_test, prediction_st)
re_st = recall_score(y_test, prediction_st)
acc_skl = accuracy_score(y_test, prediction_skl)
re_skl = recall_score(y_test, prediction_skl)
print([acc_st, acc_skl])
[re_st, re_skl]


[0.8823529411764706, 0.9019607843137255]


[0.0, 0.0]